In [ ]:
import pandas as pd
from time import sleep
import random
import os
from pytrends.request import TrendReq as UTrendReq
import requests
import json

# header copied from the development tools 
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "it-IT,it;q=0.9",
    "Cookie": "__utma=10102256.1011286115.1746523015.1746523023.1746523023.1; __utmb=10102256.10.9.1746523413327; __utmz=10102256.1746523023.1.1.utmcsr=trends.google.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmc=10102256; __utmt=1; SIDCC=AKEyXzVZfzKym6FdxJXlVCvRCWaj_FWQK1vK3u298t3Jcti6NMzJiMmcaxczE03OvV3CRqyeig0; __Secure-1PSIDCC=AKEyXzVzCRb0HeRHi8zQaX9w8wugMd3FAeoSOTLAUWlle1WmqfkFyhew250tSf-sN-BVc0fioIQk; __Secure-3PSIDCC=AKEyXzURg5XJfZU-YOmnAj9hLKECto42uEkN-tcXXFykNZmebAiazAiiPWpDuNMI2tbRIZIQrE0; _ga=GA1.3.1011286115.1746523015; _ga_VWZPXDNJJB=GS2.1.s1746523014$o1$g1$t1746523413$j0$l0$h0; _gat_gtag_UA_4401283=1; _gid=GA1.3.1881250004.1746523015; __Secure-1PSIDTS=sidts-CjIBjplskKfHCRZqZvj0kyN3JwQz97EIvM9uH7EPK8Oc4z10hkcnDv8yn5XrDhF1wi3skRAA; __Secure-3PSIDTS=sidts-CjIBjplskKfHCRZqZvj0kyN3JwQz97EIvM9uH7EPK8Oc4z10hkcnDv8yn5XrDhF1wi3skRAA; NID=523=efVNSrPoQ00SlTxMa3klk-dkBwicbtnfAEF1HXmx995n7hFvZe6r_hC4mJ5qgjxm4FDJUNo1gF0-eoyANb6mSZEp81QQ37UuJcq27LI7D9PsZuOBBxzV1GO_6GW6s6n8bMAWOz1ij506ZiMHl_Q1TyjlRD81bfoe1wCtRgLCMPlGDgdsHF69YZpbNC5wA4Lvu0TwHGOMG5M7Y3ZWRv92fCCuUJshf0M8NOkZ_B6GF4IDG6OTQ16Wm_XZ15gSBsXkcqRbnyrkXdb0T4NK1gLpLwSSDQ; OTZ=8070317_48_52_123900_48_436380; AEC=AVcja2f_ifmNbesIOG9IzwCQbyROkweSegQJNrMxwceaG-r-AjuqoYhpTtw; APISID=5wNdHApMf-f5n31a/AiolloEEV7x8HTfAl; HSID=ArCZAUAozzDDtaOHM; SAPISID=NbscY4nnm5sJTRxx/Asnp1yi9dfmpvBXyY; SID=g.a000wgiKsxsCdNDk3F4JaKgGmjEtAF_9sSJtFpdkh_PkfiyqExxypTtMX0xuknw2vUycXidY_AACgYKAS4SARYSFQHGX2MiBKonK4Cc8Nf9bRNWZpUWDRoVAUF8yKor5b8lEIMKmst35A8bg5de0076; SSID=AM10ykND0JXRqvxyy; __Secure-1PAPISID=NbscY4nnm5sJTRxx/Asnp1yi9dfmpvBXyY; __Secure-1PSID=g.a000wgiKsxsCdNDk3F4JaKgGmjEtAF_9sSJtFpdkh_PkfiyqExxyj5LGMexgmOK6vFjWvlHgmwACgYKAYoSARYSFQHGX2MidvVZzmGU99_iF1XJW8dB6BoVAUF8yKpTEgbtphSVUOrNUA50GibS0076; __Secure-3PAPISID=NbscY4nnm5sJTRxx/Asnp1yi9dfmpvBXyY; __Secure-3PSID=g.a000wgiKsxsCdNDk3F4JaKgGmjEtAF_9sSJtFpdkh_PkfiyqExxycpKYlJa8Tkq1x3duIyICgwACgYKAfQSARYSFQHGX2MiDZ3nMXBRVXZmSeCY7NT2wxoVAUF8yKoXGQd-oZ_knw7t62KgZoKA0076; OGPC=19031728-1:; SEARCH_SAMESITE=CgQI2p0B; SOCS=CAISHAgCEhJnd3NfMjAyNDA1MDgtMF9SQzEaAml0IAEaBgiA9o-yBg",
    "Referer": "https://trends.google.com/",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.4 Safari/605.1.15"}

# class of TrendReq to use the header - 
class TrendReq(UTrendReq):
    def _get_data(self, url, method='get', trim_chars=0, **kwargs):
        response = requests.request(method, url, headers=headers, timeout=self.timeout, **kwargs)
        # recognize the error 429 (blocked from google) before proceeding 
        if response.status_code == 429:
            raise Exception("429 Too Many Requests")
        # try to parse as a json file 
        text = response.text[trim_chars:]
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            print("error decoding JSON")
            print(text[:300])
            raise

# keywords per dimension 
keywords_dict = {
    "income": ["desempleo", "crisis", "pobreza"],
    "access_to_health_services": ["seguro médico", "enfermedad", "hospital"],
    "educational_lag": ["analfabetismo", "acceso a la educación", "abandono escolar"],
    "access_to_social_security": ["seguridad social", "pensiones", "prestaciones"],
    "housing": ["vivienda", "hacinamiento", "servicios básicos"],
    "access_to_food": ["inseguridad alimentaria", "hambre", "comida"],
    "social_cohesion": ["violencia", "discriminación", "exclusión social"]}

# states 
region_codes = [ "MX-GUA", "MX-MIC", "MX-SIN", "MX-CHH", "MX-GRO",
                "MX-TAM", "MX-BCN", "MX-ZAC", "MX-COL", "MX-JAL"]

# time 
timeframe = '2022-01-01 2022-12-31'

# start PYTRENDS with the personalized class 
pytrends = TrendReq(hl='es-MX', tz=360, timeout=(10, 30))

# do the research per state*dimension 
for region_code in region_codes:
    for dimension, kw_list in keywords_dict.items():
        filename = f"trends_{region_code}_{dimension}.csv"
        success = False
        while not success:
            try:
                pytrends.build_payload(
                    kw_list=kw_list,
                    cat=0,
                    timeframe=timeframe,
                    geo=region_code,
                    gprop='')
                iot = pytrends.interest_over_time()

                if iot.empty:
                    print(f"no data for {region_code} - {dimension}")
                    break

                if 'isPartial' in iot.columns:
                    iot.drop(columns='isPartial', inplace=True)

                rename_dict = {
                    kw: f"{kw} ({dimension}) [{region_code}]"
                    for kw in kw_list}
                iot.rename(columns=rename_dict, inplace=True)
                iot.to_csv(filename)
                print(f"saved {filename}")
                success = True
                sleep(random.randint(20, 60))

            except Exception as e:
                if "429" in str(e):
                    print(f"blocked with error 429 {region_code} - {dimension}...")
                    sleep(30)
                else:
                    print(f"error for {region_code} - {dimension}: {e}")
                    sleep(60)
                    break